Day 3 – SQL via Python: NYC School Data Exploration

In [36]:
import pandas as pd
import psycopg2




In [39]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()



School Distribution — Schools per Borough

In [47]:
df = pd.read_sql("SELECT * FROM nyc_schools.school_demographics LIMIT 5;", engine)
print(df.columns.tolist())

['dbn', 'Name', 'schoolyear', 'fl_percent', 'frl_percent', 'total_enrollment', 'prek', 'k', 'grade1', 'grade2', 'grade3', 'grade4', 'grade5', 'grade6', 'grade7', 'grade8', 'grade9', 'grade10', 'grade11', 'grade12', 'ell_num', 'ell_percent', 'sped_num', 'sped_percent', 'ctt_num', 'selfcontained_num', 'asian_num', 'asian_per', 'black_num', 'black_per', 'hispanic_num', 'hispanic_per', 'white_num', 'white_per', 'male_num', 'male_per', 'female_num', 'female_per']


In [50]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, engine)
df.head()



,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [56]:
# Count schools by borough
pd.read_sql("""
SELECT
  borough,
  COUNT(DISTINCT dbn) AS total_schools
FROM nyc_schools.high_school_directory
GROUP BY borough
ORDER BY total_schools DESC;
""", engine)

,borough,total_schools
0,Brooklyn,121
1,Bronx,118
2,Manhattan,106
3,Queens,80
4,Staten Island,10


 2. Average % of English Language Learners (ELL) per borough


In [ ]:
The query combines the high school directory with the demographics data and finds the average percentage of English Language Learners (ELL) in each borough.

In [72]:

query = """
SELECT
    h.borough,
    ROUND(AVG(COALESCE(d.ell_percent, 0))::numeric, 2) AS avg_ell_percent,
    COUNT(d.ell_percent) AS rows_with_ell
FROM nyc_schools.high_school_directory h
LEFT JOIN nyc_schools.school_demographics d USING (dbn)
GROUP BY h.borough
ORDER BY h.borough;
"""

ell_per_borough = pd.read_sql(query, engine)
print(ell_per_borough)


 






         borough  avg_ell_percent  rows_with_ell
0          Bronx             0.00              0
1       Brooklyn             0.00              0
2      Manhattan             2.18             40
3         Queens             0.00              0
4  Staten Island             0.00              0


Top-3 schools per borough by special-education share

In [75]:
query = """
SELECT borough, school_name, dbn, ROUND(sped_percent::numeric, 2) AS sped_percent
FROM (
  SELECT
    h.borough, h.school_name, h.dbn,
    MAX(d.sped_percent) AS sped_percent,
    DENSE_RANK() OVER (
      PARTITION BY h.borough
      ORDER BY AVG(d.sped_percent) DESC NULLS LAST
    ) AS rk
  FROM nyc_schools.high_school_directory h
  JOIN nyc_schools.school_demographics d USING (dbn)
  GROUP BY h.borough, h.school_name, h.dbn
) s
WHERE rk <= 3
ORDER BY borough, sped_percent DESC;
"""

top_sped_schools = pd.read_sql(query, engine)
print(top_sped_schools)

     borough                                    school_name     dbn  \
0  Manhattan                     East Side Community School  01M450   
1  Manhattan                        Marta Valle High School  01M509   
2  Manhattan  Henry Street School for International Studies  01M292   

   sped_percent  
0          28.8  
1          25.9  
2          25.1  


Insight